# Bloomberg BQuant Webinar Series: Intro to BQuant

This is a companion notebook to the Bloomberg [Intro to BQuant](https://blinks.bloomberg.com/screens/PLYR%20VOD%20329737029) webinar.  

In [ ]:
import bql
from ipywidgets import Button, VBox, HBox, Layout
from IPython.display import display
from bqwidgets import TickerAutoComplete, DataGrid
import pandas as pd

In [ ]:
def merge_dfs(response):
    all_dfs = [item.df()[item.name] for item in response]
    return pd.concat(all_dfs, axis=1)

In [ ]:
bq = bql.Service()

In [ ]:
input_ticker = TickerAutoComplete(value = 'UKX Index', yellow_keys = ['Index'], layout=Layout(width='150px'))
run_button = Button(description = 'Run', layout=Layout(width='100px'))
result_box = VBox([], layout=Layout(width='500px'))

In [ ]:
def get_data(caller):
    universe = bq.univ.members(input_ticker.value)
    fields= {'Name' : bq.data.name(),
             'Price' : bq.data.px_last(),
             'P/E' : bq.data.pe_ratio(fill='prev')}
    request = bql.Request(universe, fields)
    resp = bq.execute(request)
    column_defs = [{'headerName' : 'Ticker', 'field':'ID', 'width': 150},
               {'headerName' : 'Name', 'field':'Name', 'width': 150},
               {'headerName' : 'Price', 'field':'Price', 'width': 80},
               {'headerName' : 'P/E', 'field':'P/E', 'width': 80}]
    data = merge_dfs(resp)
    dgrid = DataGrid(data=data.round(2).reset_index(), column_defs=column_defs)
    result_box.children = [dgrid]
run_button.on_click(get_data)

In [ ]:
ui = VBox(layout=Layout(width='600px'))
ui.children = [HBox([input_ticker, run_button]), result_box]
display(ui)